# Review statistics from logs

In [16]:
from pathlib import Path
from datetime import datetime
import ast
import json
import pandas as pd
import re

In [136]:
def extract_session_info(p2log, cleanup=True):
    
    new_session = re.compile(r"^New Session: (?P<timestamp_str>\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d)$")
    game_info = re.compile(r"^(?P<time_stamp>\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d):   update info for gameidx: \d+ and name (?P<game_name>.*)$")
    host_info = re.compile(r"^(?P<yr>\d\d\d\d)-(?P<mth>\d\d)-(?P<day>\d\d) \d\d:\d\d:\d\d:    Host Index: \d+ Host Name: (?P<host_name>.*)$")
    cast_info = re.compile(r"^(?P<yr>\d\d\d\d)-(?P<mth>\d\d)-(?P<day>\d\d) \d\d:\d\d:\d\d:    player idxs: .* players: (?P<players>.*), audience: \d+$")

    cols = ['session', 'session_ts', 'game', 'host', 'players']
    df = pd.DataFrame(columns=cols)

    with p2log.open() as f:
        i = 1
        session = 0
        new_row = pd.DataFrame(columns=cols)

        while True:
            line = f.readline()
            i += 1                   
            # search for new session
            m = new_session.search(line)
            if m:
                session += 1
                session_ts = m.group('timestamp_str')
                continue
            
            # search for game and cast info
            m = game_info.search(line)
            if m:
                # append last game info to df, unless it is empty
                if not new_row.empty:
                    df = new_row if df.empty else pd.concat([df, new_row], ignore_index=False) 
                
                game_name = m.group('game_name')
                game_ts = m.group('time_stamp')
                new_row = pd.DataFrame(
                    data={ 'session': [session], 'session_ts': [session_ts], 'game': [game_name], 'host': [None], 'players': [None]},
                    index=[datetime.fromisoformat(game_ts)]
                    )
                continue
            m = host_info.search(line)
            if m:
                host_name = m.group('host_name')
                new_row['host'] = host_name
                continue
            m = cast_info.search(line)
            if m:
                players = ast.literal_eval(m.group('players'))
                new_row.iloc[0,4] = ','.join(players)
                continue
            # test for EOF
            if line == '': break
            
    if cleanup:
        print('cleaning up too short sessions')
        sessions = df.session.unique()
        for s in sessions:
            session_df = df.loc[df.session == s]
            session_idxs = session_df.index
            first, last = session_idxs[0], session_idxs[-1]
            session_duration = (last - first).total_seconds()
            if session_duration < 10 * 60:
                df = df.drop(session_idxs)
        
    return df

In [143]:
p2log = Path('..') / 'logs/_short.log'
assert p2log.exists()
df = extract_session_info(p2log)
sessions = df.session.unique()
print(f"{len(sessions)} valid sessions")
print(f"{len(df)} games")
df.head(5)


cleaning up too short sessions
14 valid sessions
181 games


,session,session_ts,game,host,players
2024-07-08 20:10:59,7,2024-07-08 20:08:51,Everyone Dies,Knick,"Etienne,Xu,Yirong"
2024-07-08 20:17:29,7,2024-07-08 20:08:51,Add-on fairy tale,Grace,"Etienne,Knick,Xu,Yirong"
2024-07-08 20:18:29,7,2024-07-08 20:08:51,One Three Five,Yirong,"Etienne,Grace,Knick"
2024-07-08 20:22:15,7,2024-07-08 20:08:51,A Date with Me,None,None
2024-07-08 20:26:40,7,2024-07-08 20:08:51,Last Player Standing,Etienne,"Grace,Knick,Xu,Yirong"


In [144]:
session_idx = -2

selected_session = df.session.unique()[session_idx]
df.loc[df.session == selected_session]

,session,session_ts,game,host,players
2024-11-02 16:09:58,85,2024-11-02 15:55:54,A Date with Me,Stacy,"Alex,Charles,Chris,Don,Eddie,Elva,Etienne,Fulv..."
2024-11-02 16:12:07,85,2024-11-02 15:55:54,Sure Ding,Jennel,"Don,Stacy,Yirong"
2024-11-02 16:17:01,85,2024-11-02 15:55:54,Forward Reverse,Alex,"Eddie,Fulvio,Jennel"
2024-11-02 16:21:53,85,2024-11-02 15:55:54,Questions Only,Charles,"Alex,Chris,Don,Eddie,Elva,Etienne,Fulvio,Haili..."
2024-11-02 16:25:33,85,2024-11-02 15:55:54,Stunt Doubles,Chris,"Charles,Fulvio,Haili,Meg"
2024-11-02 16:29:59,85,2024-11-02 15:55:54,Two lines,Yirong,"Charles,Eddie,Stacy"
2024-11-02 16:33:26,85,2024-11-02 15:55:54,Last Comic Standing,Harry,"Alex,Charles,Chris,Haili"
2024-11-02 16:39:59,85,2024-11-02 15:55:54,Puppets,Eddie,"Harry,Jennel"
2024-11-02 16:42:32,85,2024-11-02 15:55:54,Musical Freeze,None,None
2024-11-02 16:48:17,85,2024-11-02 15:55:54,"Story, Story, Die!",None,None
